In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt

In [ ]:
Training='/content/drive/MyDrive/letters_recognition/'

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255,validation_split=0.2,
        rotation_range=30,
        zoom_range = 0.1,
        shear_range = 0.2,
        width_shift_range=0.122,
        height_shift_range=0.122,
        brightness_range = (0.5, 1.5)
                                  )

In [ ]:
valid_datagen = ImageDataGenerator(rescale=1/255,validation_split=0.2)

In [ ]:
input_shape=(32,32,3)

In [ ]:
train_data = train_datagen.flow_from_directory(
   Training,
    subset="training",
    seed=2,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=32,
    class_mode="categorical")

Found 3626 images belonging to 17 classes.


In [ ]:
valid_data = valid_datagen.flow_from_directory(
   Training,
    subset="validation",
    seed=2,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=32,
    class_mode="categorical")

Found 899 images belonging to 17 classes.


In [ ]:
## imports for model
from tensorflow import keras
from keras.layers import Dropout, Flatten,Activation ,Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
from tensorflow.keras.applications import ResNet50,DenseNet121,MobileNetV2


In [ ]:
MobileNetV2=MobileNetV2(include_top=False,input_shape=(32, 32, 3))
MobileNetV2.trainable=False

In [ ]:
Early_Stopping = EarlyStopping(
    monitor='val_loss', # Same as the
    patience=5, # Wait for 5 epochs before stopping the training
    verbose=1,
    restore_best_weights='True', # Restore model weights from the epoch with the best value of the monitored quantity.
    min_delta=0.1
)
reduce_on_plateau =ReduceLROnPlateau(
        monitor  = 'loss',
        factor   = 0.1,
        patience = 2,
        verbose  = 1,
        mode     = 'min',
        epsilon  = 0.01,
        cooldown = 0,
        min_lr   = 0
    )

In [ ]:
inputs = tf.keras.Input(shape=input_shape)
x=MobileNetV2(inputs)
x = layers.Flatten()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(17, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 1, 1, 1280)       2257984   
 ional)                                                          
                                                                 
 flatten (Flatten)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1024)              1311744   
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_2 (Dense)             (None, 1024)              1049600   
                                                             

In [ ]:
History = model.fit(train_data,validation_data =valid_data,epochs = 150,batch_size =256,callbacks=[reduce_on_plateau])

Epoch 1/150
114/114 [==============================] - 53s 153ms/step - loss: 0.7888 - accuracy: 0.8489 - val_loss: 0.5583 - val_accuracy: 0.8042 - lr: 0.0010
Epoch 2/150
114/114 [==============================] - 12s 105ms/step - loss: 0.5989 - accuracy: 0.8762 - val_loss: 0.3905 - val_accuracy: 0.8799 - lr: 0.0010
Epoch 3/150
114/114 [==============================] - 11s 97ms/step - loss: 0.4314 - accuracy: 0.9117 - val_loss: 0.8273 - val_accuracy: 0.8387 - lr: 0.0010
Epoch 4/150
114/114 [==============================] - 12s 104ms/step - loss: 0.4656 - accuracy: 0.9040 - val_loss: 0.6017 - val_accuracy: 0.8877 - lr: 0.0010
Epoch 5/150
114/114 [==============================] - ETA: 0s - loss: 0.4485 - accuracy: 0.9046
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
114/114 [==============================] - 12s 102ms/step - loss: 0.4485 - accuracy: 0.9046 - val_loss: 2.6217 - val_accuracy: 0.5984 - lr: 0.0010
Epoch 6/150
114/114 [=======================

In [ ]:
model.save('/content/drive/MyDrive/model_letters.h5')

In [ ]:
letters={0:'A',1:'B',2:'G',3:'D',4:'R',5:'S',6:'C',7:'T',8:'E',9:'F',10:'K',11:'L',12:'M',13:'N',14:'H',15:'W',16:'Y'}